<a href="https://colab.research.google.com/github/vinay-852/Video_Synopsis/blob/main/Video_Synopsis(Clear).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Copy data without downloading.......
import os
import requests
import tqdm

def text_retriever(url) -> list[str]:
  """
  Retrieves URLs from a text file.

  Args:
      url (str): The URL of the text file containing video URLs.

  Returns:
      list[str]: A list of video URLs extracted from the text file.

  Raises:
      Exception: If there's an error retrieving or parsing the text file.
  """

  try:
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception for non-2xx status codes

    text = response.text
    urls = text.splitlines()  # Split text into lines, removing unnecessary spaces

    return urls
  except Exception as e:
    print(f"Error retrieving URLs: {e}")
    return []

In [ ]:
urls=text_retriever("https://raw.githubusercontent.com/Kitware/MEVID/refs/heads/main/mevid-v1-video-URLS.txt")

In [ ]:
#Download Dataset form internet.........
import os
import requests
from tqdm import tqdm

def download_file(url, save_dir):
  """
  Download a file from a given URL and save it to a specified directory.

  Args:
    url: The URL of the file to download.
    save_dir: The directory where the file should be saved.

  Returns:
    None
  """
  try:
      if not os.path.exists(save_dir):
          os.makedirs(save_dir)

      filename = url.split('/')[-1]
      file_path = os.path.join(save_dir, filename)

      response = requests.get(url, stream=True)
      total_size = int(response.headers.get('content-length', 0))

      with open(file_path, 'wb') as f:
          with tqdm(total=total_size, unit='B', unit_scale=True, desc='Downloading') as progress_bar:
              for data in response.iter_content(chunk_size=1024):
                  if data:
                      progress_bar.update(len(data))
                      f.write(data)

      print(f"Download complete! File saved to: {file_path}")
  except Exception as e:
      print(f"Error downloading file: {e}")

In [ ]:
download_file(urls[0], "/content/dataset")

Downloading: 100%|██████████| 162M/162M [00:18<00:00, 8.71MB/s]

Download complete! File saved to: /content/dataset/2018-05-16.14-25-01.14-30-01.school.G639.r13.avi


In [ ]:
!pip install -q supervision
!pip install -q git+https://github.com/THU-MIG/yolov10.git
!pip install deep-sort-realtime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 26.1 MB/s eta 0:00:00


In [ ]:
download_file("https://github.com/jameslahm/yolov10/releases/download/v1.0/yolov10l.pt","/content/weights")

Downloading: 100%|██████████| 104M/104M [00:13<00:00, 7.99MB/s]

Download complete! File saved to: /content/weights/yolov10l.pt


In [ ]:
#Video Format which can be played in Colab..............
from IPython.display import HTML
from base64 import b64encode
def play_video(filename):
  html = ''
  video = open(filename,'rb').read()
  src = 'data:video/mp4;base64,' + b64encode(video).decode()
  html += fr'<video width=900 controls autoplay loop><source src="%s" type="video/mp4"></video>' % src
  return HTML(html)

In [ ]:
#Display on Colab ........
from IPython.display import HTML
from base64 import b64encode
import subprocess

def process_and_play_video(input_filename, output_filename):
  """
  Uses the ffmpeg command to process a video file and play it in Colab.

  Args:
      input_filename (str): The path to the input video file.
      output_filename (str): The path to save the processed video file.

  Returns:
      HTML: An HTML element to display the processed video in Colab.
  """
  subprocess.run([
        'ffmpeg',
        '-hide_banner',
        '-loglevel', 'error',
        '-i', input_filename,
        '-vcodec', 'libx264',
        output_filename,
        '-y'
    ])

  return play_video(output_filename)

In [ ]:
import cv2
import datetime
from absl import app, flags
from absl.flags import FLAGS
from deep_sort_realtime.deepsort_tracker import DeepSort
import os
import csv
import numpy as np
from typing import Tuple, List
import torch
from tqdm import tqdm

In [ ]:
#YOLOv10 model Operations..........
from typing import Tuple, List
import numpy as np
from ultralytics import YOLOv10

# Load YOLO model and COCO classes
def load_yolo_model(model_file, class_file="coco.names"):
    """
    Loads the YOLO model for object detection and the class names from the COCO dataset.

    Args:
        model_file (str): Path to the YOLO model file.
        class_file (str): Path to the file containing COCO class names.

    Returns:
        tuple:
            - model (YOLOv10): The loaded YOLO model.
            - class_names (list): List of class names corresponding to COCO dataset classes.
    """
    # Check if a GPU is available; if not, use the CPU
    device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
    print(f'Using device: {device}')

    # Load the YOLO model
    model = YOLOv10(model_file)

    # Load COCO class names from the specified file
    class_names=text_retriever("https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names")

    return model, class_names

def detect_objects(model, frame, conf_threshold):
    """
    Performs object detection on a given frame using the YOLO model.

    Args:
        model (YOLOv10): The loaded YOLO model used for detecting objects.
        frame (numpy.ndarray): The input video frame to perform object detection on.
        conf_threshold (float): Confidence threshold to filter weak detections.

    Returns:
        list: A list of detections where each detection contains bounding box, confidence, and class ID.
    """
    # Perform object detection on the frame
    results = model(frame, verbose=False)[0]

    detections = []
    for det in results.boxes:
        confidence = det.conf
        label = det.cls
        bbox = det.xyxy[0]  # Bounding box coordinates: [x1, y1, x2, y2]
        x1, y1, x2, y2 = map(int, bbox)
        class_id = int(label)

        # Filter out weak detections based on confidence threshold
        if confidence >= conf_threshold:
            detections.append([[x1, y1, x2 - x1, y2 - y1], confidence, class_id])

    return detections

In [ ]:
download_file("https://i.cdn.newsbytesapp.com/images/l71620241008162609.jpeg", "/content/")

Downloading: 100%|██████████| 132k/132k [00:00<00:00, 3.19MB/s]

Download complete! File saved to: /content/l71620241008162609.jpeg


In [ ]:
model, class_names =load_yolo_model("/content/weights/yolov10l.pt")

/usr/local/lib/python3.10/dist-packages/ultralytics/nn/tasks.py:733: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(file, map_location="cpu")


Using device: cpu


In [ ]:
detections = detect_objects(model, '/content/l71620241008162609.jpeg', conf_threshold=0.5)

In [ ]:
print(detections)

[[[886, 78, 274, 968], tensor([0.9338]), 0], [[1477, 433, 136, 341], tensor([0.8929]), 0], [[941, 19, 306, 358], tensor([0.8528]), 34], [[1649, 543, 53, 138], tensor([0.8219]), 0], [[1745, 562, 47, 102], tensor([0.7588]), 0], [[1829, 554, 47, 122], tensor([0.7557]), 0], [[1874, 544, 45, 135], tensor([0.7498]), 0], [[1465, 526, 48, 53], tensor([0.7447]), 35], [[736, 583, 44, 73], tensor([0.5945]), 0], [[434, 573, 37, 82], tensor([0.5820]), 0], [[815, 592, 38, 64], tensor([0.5511]), 0]]


In [ ]:
import cv2
import numpy as np

def generate_background(video_file, output_file='background.jpg', method='median', num_frames=30):
    """
    Generate a background image from the first 'num_frames' frames of a video by averaging or taking the median.

    Args:
        video_file (str): Path to the input video file.
        output_file (str): Path where the background image will be saved.
        method (str): Method to generate background ('mean' or 'median').
        num_frames (int): Number of frames to use for background generation.

    Returns:
        None
    """
    # Open the video file
    cap = cv2.VideoCapture(video_file)

    # Check if the video was opened successfully
    if not cap.isOpened():
        print("Error: Could not open video file.")
        return

    # Initialize a list to store frames
    frames = []
    frame_count = 0

    # Read frames from the video until num_frames are collected
    while frame_count < num_frames:
        ret, frame = cap.read()
        if not ret:
            print(f"End of video reached at frame {frame_count}.")
            break

        # Append the frame to the list
        frames.append(frame)
        frame_count += 1

    # Release the video capture object
    cap.release()

    # Convert the list of frames to a numpy array
    frames = np.array(frames)

    # Generate the background by computing the median or mean of the frames
    if method == 'median':
        background = np.median(frames, axis=0).astype(dtype=np.uint8)  # Median
    elif method == 'mean':
        background = np.mean(frames, axis=0).astype(dtype=np.uint8)  # Mean
    else:
        print("Error: Method must be 'mean' or 'median'.")
        return

    # Save the generated background as an image file
    cv2.imwrite(output_file, background)
    print(f"Background image saved as {output_file}")

# Example usage:
video_file = '/content/dataset/2018-05-16.14-25-01.14-30-01.school.G639.r13.avi'  # Replace with the path to your video file
generate_background(video_file, output_file='background.jpg', method='median', num_frames=30)


Background image saved as background.jpg


In [ ]:
#Deepsort Operations...........
import numpy as np
from deep_sort_realtime.deepsort_tracker import DeepSort

def initialize_deepsort(max_age=20, n_init=3):
    """
    Initializes the DeepSort tracker.

    Args:
        max_age (int): Maximum number of frames to keep alive a track.
        n_init (int): Minimum number of detections before a track is confirmed.

    Returns:
        DeepSort: An instance of the DeepSort tracker.
    """
    return DeepSort(max_age=max_age, n_init=n_init)

def update_tracker(tracker, detections, frame):
    """
    Updates the DeepSort tracker with new detections.

    Args:
        tracker (DeepSort): The DeepSort tracker instance.
        detections (list): List of detections in the format [bounding_box, confidence, class_id].
        frame (numpy.ndarray): The current video frame.

    Returns:
        List: Updated tracks from the DeepSort tracker.
    """
    return tracker.update_tracks(detections, frame=frame)


In [ ]:
#Useful Utility function to complete tracking video.........
import os
import cv2
import csv
import numpy as np
import torch
from deep_sort_realtime.deepsort_tracker import DeepSort
from tqdm import tqdm

# Initialize video capture and writer
def initialize_video(video_file, output_file):
    """
    Initializes video capture and writer objects for reading from the input video and writing to the output video.

    Args:
        video_file (str): Path to the input video file.
        output_file (str): Path to the output video file where the processed video will be saved.

    Returns:
        tuple:
            - video_cap (cv2.VideoCapture): The video capture object.
            - writer (cv2.VideoWriter): The video writer object for saving processed frames.
            - total_frames (int): Total number of frames in the input video.
            - frame_width (int): Width of each frame in the video.
            - frame_height (int): Height of each frame in the video.
    """
    video_cap = cv2.VideoCapture(video_file)
    frame_width = int(video_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(video_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(video_cap.get(cv2.CAP_PROP_FPS))

    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    writer = cv2.VideoWriter(output_file, fourcc, fps, (frame_width, frame_height))

    total_frames = int(video_cap.get(cv2.CAP_PROP_FRAME_COUNT))  # Total frames for tqdm
    return video_cap, writer, total_frames, frame_width, frame_height


def extract_hsv(frame, ltrb):
    """
    Extracts the mean hue, saturation, and value (HSV) from the region of interest (ROI) defined by the bounding box.

    Args:
        frame (numpy.ndarray): The current video frame.
        bbox (tuple): Bounding box coordinates (x1, y1, x2, y2) defining the ROI.

    Returns:
        tuple:
            - mean_hue (float): Mean hue value in the ROI.
            - mean_saturation (float): Mean saturation value in the ROI.
            - mean_value (float): Mean value (brightness) in the ROI.
    """
    x1, y1, x2, y2 = map(int, ltrb)
    # Ensure the bounding box coordinates are within the frame boundaries
    x1, y1, x2, y2 = max(0, x1), max(0, y1), min(frame.shape[1], x2), min(frame.shape[0], y2)

    if x2 > x1 and y2 > y1:
        # Extract the region of interest (ROI) based on the bounding box
        object_roi = frame[y1:y2, x1:x2]

        # Convert the ROI to HSV color space
        hsv_roi = cv2.cvtColor(object_roi, cv2.COLOR_BGR2HSV)

        # Compute the mean values for hue, saturation, and value in the HSV space
        mean_hue = np.mean(hsv_roi[:, :, 0])
        mean_saturation = np.mean(hsv_roi[:, :, 1])
        mean_value = np.mean(hsv_roi[:, :, 2])

        return mean_hue, mean_saturation, mean_value
    return None, None, None

# Write tracking data to CSV
def write_csv(writer_csv, track_id, class_name, frame_count, bbox, hue, saturation, value):
    """
    Writes the tracking data for an object to the CSV file, including track ID, class, frame number, bounding box, and HSV color information.

    Args:
        writer_csv (csv.writer): CSV writer object to save tracking data.
        track_id (int): Unique ID for the tracked object.
        class_name (str): Name of the detected object class.
        frame_count (int): Current frame number.
        bbox (tuple): Bounding box coordinates (x1, y1, x2, y2).
        hue (float): Mean hue value of the object in the ROI.
        saturation (float): Mean saturation value of the object in the ROI.
        value (float): Mean value (brightness) of the object in the ROI.
    """
    writer_csv.writerow([track_id, class_name, frame_count, bbox, hue, saturation, value])

In [ ]:
import csv
import cv2
import numpy as np
from tqdm import tqdm

def calculate_frame_difference(prev_frame, curr_frame):
    """
    Calculate the difference between two frames using Mean Squared Error (MSE).

    Args:
        prev_frame (np.array): The previous frame.
        curr_frame (np.array): The current frame.

    Returns:
        float: The difference between the two frames.
    """
    # Convert frames to grayscale for simplicity
    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    curr_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)

    # Compute Mean Squared Error (MSE) between the two frames
    diff = np.sum((prev_gray.astype("float") - curr_gray.astype("float")) ** 2)
    diff /= float(prev_gray.shape[0] * prev_gray.shape[1])

    return diff



In [ ]:
import os
import cv2
import csv
import numpy as np
from tqdm import tqdm
from deep_sort_realtime.deepsort_tracker import DeepSort

def extract_object_tracks(model_file, video_file, output_file, conf_threshold, threshold, crops_dir="object_crops"):
    """
    Main function to extract object tracks from key frames of a video using YOLO for object detection and
    DeepSort for tracking. The function saves the tracking data, including bounding boxes, HSV values, and
    cropped objects to a CSV file, and generates an output video with annotated tracks.

    Args:
        model_file (str): Path to the YOLO model file.
        video_file (str): Path to the input video file.
        output_file (str): Path to the output video file to save the annotated video.
        conf_threshold (float): Confidence threshold to filter weak detections.
        threshold (float): Threshold for selecting key frames based on frame difference.
        crops_dir (str): Directory where cropped object images will be saved.
    """
    try:
        # Initialize video capture, writer, and other video properties
        video_cap, writer, total_frames, frame_width, frame_height = initialize_video(video_file, output_file)

        # Load YOLO model and COCO class names
        model, class_names = load_yolo_model(model_file)

        # Initialize DeepSort tracker
        tracker = DeepSort(max_age=20, n_init=3)

        # Create directory for saving crops if it doesn't exist
        if not os.path.exists(crops_dir):
            os.makedirs(crops_dir)

        # Open CSV file for saving object tracks
        with open('object_tracks.csv', mode='w', newline='') as file:
            writer_csv = csv.writer(file)
            writer_csv.writerow(['Track ID', 'Class Name', 'Frame', 'Bounding Box', 'Hue', 'Saturation', 'Value', 'Crop Path'])

            frame_count = 0
            prev_frame = None

            # Progress bar for processing video frames
            with tqdm(total=total_frames, desc="Processing Frames") as pbar:
                while True:
                    ret, frame = video_cap.read()
                    frame_count += 1
                    if not ret:
                        print("End of video.")
                        break

                    # If this is the first frame, store it and continue to the next
                    if prev_frame is None:
                        prev_frame = frame
                        continue

                    # Calculate the difference between the current frame and the previous frame
                    frame_diff = calculate_frame_difference(prev_frame, frame)

                    # Check if the difference exceeds the threshold (key frame selection)
                    if frame_diff > threshold:
                        print(f"Processing key frame {frame_count} (difference: {frame_diff})")

                        # Object detection
                        detections = detect_objects(model, frame, conf_threshold)
                        tracks = update_tracker(tracker, detections, frame)

                        for track in tracks:
                            if track.is_confirmed():
                                track_id = track.track_id
                                ltrb = track.to_ltrb()  # Bounding box format: (left, top, right, bottom)
                                class_id = track.get_det_class()
                                class_name = class_names[class_id]

                                # Extract HSV color features from the bounding box region
                                hue, saturation, value = extract_hsv(frame, ltrb)

                                # Crop the object and save the cropped image
                                x1, y1, x2, y2 = map(int, ltrb)
                                crop_img = frame[y1:y2, x1:x2]
                                crop_path = f"{crops_dir}/track_{track_id}_frame_{frame_count}.png"
                                cv2.imwrite(crop_path, crop_img)

                                if hue is not None:
                                    # Write the tracking data (including HSV and crop path) to the CSV file
                                    writer_csv.writerow([track_id, class_name, frame_count, ltrb, hue, saturation, value, crop_path])

                                    # Draw bounding box and label on the frame
                                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                                    cv2.putText(frame, f'{class_name} {track_id}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

                        # Write the frame with annotations to the output video
                        writer.write(frame)

                    # Update the previous frame
                    prev_frame = frame
                    pbar.update(1)

        # Release video capture and writer resources
        video_cap.release()
        writer.release()
        print(f"Tracking completed and saved to 'object_tracks.csv'. Cropped images saved to '{crops_dir}'.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage of the function with keyframe selection and cropping
model_file = "weights/yolov10l.pt"
video_file = "/content/dataset/2018-05-16.14-25-01.14-30-01.school.G639.r13.avi"
output_file = "output_tracked_keyframes.mp4"
conf_threshold = 0.5
frame_diff_threshold = 6  # Set a threshold for keyframe selection

# Call the main function to extract object tracks based on key frames and crop objects
extract_object_tracks(model_file, video_file, output_file, conf_threshold, frame_diff_threshold)


Using device: cpu


/usr/local/lib/python3.10/dist-packages/ultralytics/nn/tasks.py:733: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(file, map_location="cpu")
/usr/local/lib

Processing key frame 61 (difference: 12.92500048585199)
Processing key frame 62 (difference: 16.59521047108209)


Processing Frames:   1%|▏         | 118/9001 [00:10<05:14, 28.23it/s]

Processing key frame 121 (difference: 15.31318116449005)
Processing key frame 122 (difference: 16.554047632929105)


...

Processing key frame 4535 (difference: 31.923894686722637)


Processing Frames:  50%|█████     | 4534/9001 [35:33<2:15:24,  1.82s/it]

Processing key frame 4536 (difference: 30.83566493703358)


Processing Frames:  50%|█████     | 4534/9001 [35:34<35:03,  2.12it/s]  

An error occurred: OpenCV(4.10.0) /io/opencv/modules/imgcodecs/src/loadsave.cpp:798: error: (-215:Assertion failed) !_img.empty() in function 'imwrite'



In [ ]:
import csv
import math

def calculate_area_velocity_tubes(input_csv, output_csv):
    """
    Calculate area and velocity for each object in the object_tracks.csv file by considering objects in the same tube
    (i.e., same Track ID) and computing the velocity based on the movement between consecutive frames.

    Args:
        input_csv (str): Path to the input CSV file (object_tracks.csv).
        output_csv (str): Path to the output CSV file to save the results with area and velocity.
    """
    try:
        rows = []

        # Read the input CSV file
        with open(input_csv, 'r') as file:
            reader = csv.DictReader(file)
            for row in reader:
                # Convert frame to int and bounding box values to float for calculations
                row['Frame'] = int(row['Frame'])

                # Parse the bounding box values by removing brackets and splitting by commas or spaces
                bbox_str = row['Bounding Box'].strip('[]')
                row['Bounding Box'] = tuple(map(float, bbox_str.replace(",", " ").split()))

                rows.append(row)

        # Sort rows by Track ID and Frame to ensure sequential processing
        rows.sort(key=lambda x: (int(x['Track ID']), x['Frame']))

        # Open the output CSV file for writing
        with open(output_csv, 'w', newline='') as file:
            writer = csv.writer(file)
            # Write the new header with Area and Velocity columns
            writer.writerow(['Track ID', 'Class Name', 'Frame', 'Bounding Box', 'Hue', 'Saturation', 'Value', 'Area', 'Velocity'])

            prev_objects = {}  # To store the last seen object positions per Track ID

            # Loop through the rows and calculate area and velocity
            for row in rows:
                try:
                    # Extract bounding box values
                    left, top, right, bottom = row['Bounding Box']

                    # Calculate area of the bounding box
                    area = (right - left) * (bottom - top)

                    # Calculate velocity for the same Track ID between consecutive frames
                    track_id = row['Track ID']
                    current_frame = row['Frame']

                    # Calculate center of the current bounding box
                    center_x = (left + right) / 2
                    center_y = (top + bottom) / 2

                    # Check if we have a previous record for the same track_id (same tube)
                    if track_id in prev_objects:
                        prev_frame, (prev_center_x, prev_center_y) = prev_objects[track_id]

                        # Calculate velocity if the object exists in the previous frame (adjacent frames only)
                        if current_frame == prev_frame + 1:
                            velocity = math.sqrt((center_x - prev_center_x) ** 2 + (center_y - prev_center_y) ** 2)
                        else:
                            velocity = 0  # If not consecutive frames, velocity is 0
                    else:
                        velocity = 0  # First appearance of the object, velocity is 0

                    # Update the previous object record with the current frame and center
                    prev_objects[track_id] = (current_frame, (center_x, center_y))

                    # Write the row to the new CSV file with calculated area and velocity
                    writer.writerow([
                        row['Track ID'], row['Class Name'], row['Frame'], row['Bounding Box'],
                        row['Hue'], row['Saturation'], row['Value'], area, velocity
                    ])

                except Exception as row_error:
                    print(f"Error processing row: {row}, error: {row_error}")
                    continue  # Skip any problematic rows

        print(f"Area and velocity calculations completed and saved to {output_csv}.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Call the function with the input and output CSV file paths
input_csv = 'object_tracks.csv'
output_csv = 'object_tracks_with_area_velocity_tubes.csv'

calculate_area_velocity_tubes(input_csv, output_csv)


Area and velocity calculations completed and saved to object_tracks_with_area_velocity_tubes.csv.


In [ ]:
import pandas as pd

df = pd.read_csv('object_tracks_with_area_velocity_tubes.csv')
df.head(50)

,Track ID,Class Name,Frame,Bounding Box,Hue,Saturation,Value,Area,Velocity
0,1,person,541,"(489.55, 1200.4, 565.39, 1388.2)",137.394807,71.200518,79.612332,14242.752,0.000000
1,1,person,542,"(490.86, 1199.1, 566.59, 1385.0)",136.710455,72.908535,78.604196,14078.207,2.576339
2,1,person,543,"(491.49, 1199.5, 568.0, 1384.9)",135.773898,73.498222,76.948506,14184.954,1.030970
3,1,person,544,"(492.17, 1200.4, 569.26, 1384.3)",135.370412,75.178289,76.172290,14176.851,0.981529
4,1,person,545,"(491.82, 1202.2, 569.42, 1384.2)",135.139405,76.280572,74.641448,14123.200,0.855292
5,1,person,546,"(493.9, 1201.4, 573.41, 1383.6)",134.511195,75.309959,75.796429,14486.722,3.114679
6,1,person,547,"(496.4, 1201.8, 577.78, 1382.8)",131.762704,74.146920,75.985540,14729.780,3.440817
7,1,person,548,"(498.14, 1201.3, 581.82, 1381.2)",130.726305,74.122289,76.373159,15054.032,3.074833
8,1,person,549,"(499.43, 1200.4, 585.87, 1380.6)",130.092506,72.653876,77.176421,15576.488,2.773337
9,1,person,550,"(501.37, 1199.4, 589.77, 1379.1)",128.673990,71.707702,77.473674,15885.480,3.176303


In [ ]:
import numpy as np
import pandas as pd
import random
import math
from copy import deepcopy

import pandas as pd

def load_tracking_database(csv_file):
  """

  """
    df = pd.read_csv(csv_file)

    # Clean up the 'Bounding Box' column and convert to list of coordinates
    # The original code assumed comma-separated values and tried to convert them to int.
    # This is updated to handle space-separated float values enclosed in brackets.
    df['Bounding Box'] = df['Bounding Box'].apply(lambda x: [float(coord) for coord in x.strip('[]').split()])

    # Filter to only include persons
    df = df[df['Class Name'] == 'person']

    # Convert dataframe to list of dictionaries
    database = df.to_dict(orient='records')
    return database

# Group the tracking data by Track ID
def group_tracks_by_id(database):
    grouped_tracks = {}
    for record in database:
        track_id = record['Track ID']
        if track_id not in grouped_tracks:
            grouped_tracks[track_id] = []
        grouped_tracks[track_id].append(record)
    return list(grouped_tracks.values())

# Utility to compute Intersection over Union (IoU)
def iou(box1, box2):
    x1, y1, x2, y2 = box1
    x1_b, y1_b, x2_b, y2_b = box2

    # Calculate intersection
    inter_x1 = max(x1, x1_b)
    inter_y1 = max(y1, y1_b)
    inter_x2 = min(x2, x2_b)
    inter_y2 = min(y2, y2_b)

    inter_area = max(0, inter_x2 - inter_x1) * max(0, inter_y2 - inter_y1)
    box1_area = (x2 - x1) * (y2 - y1)
    box2_area = (x2_b - x1_b) * (y2_b - y1_b)
    union_area = box1_area + box2_area - inter_area

    return inter_area / union_area if union_area != 0 else 0

# Temporal coherence: smoothness of bounding box motion over frames
def temporal_coherence(tracks):
    coherence_score = 0
    for track in tracks:
        for i in range(1, len(track)):
            box_prev = track[i-1]['Bounding Box']
            box_curr = track[i]['Bounding Box']
            displacement = np.linalg.norm(np.array(box_prev[:2]) - np.array(box_curr[:2]))
            coherence_score += np.exp(-displacement)
    return coherence_score

# Spatial coherence: minimize overlap between bounding boxes in the same frame
def spatial_coherence(tracks):
    overlap_penalty = 0
    frames = {}
    for track in tracks:
        for frame_info in track:
            frame = frame_info['Frame']
            box = frame_info['Bounding Box']
            if frame not in frames:
                frames[frame] = []
            frames[frame].append(box)

    for boxes in frames.values():
        for i in range(len(boxes)):
            for j in range(i + 1, len(boxes)):
                overlap_penalty += max(0, iou(boxes[i], boxes[j]) - 0.1)  # Penalize for high overlap

    return overlap_penalty

# Fitness function with temporal and spatial coherence
def fitness(tracks, w1=1.5, w2=2):
    return w1 * temporal_coherence(tracks) - w2 * spatial_coherence(tracks)

# Scale down the bounding box to reduce chances of overlap
def scale_bbox(bbox, scale_factor=0.5):
    x1, y1, x2, y2 = bbox
    width = x2 - x1
    height = y2 - y1
    center_x = (x1 + x2) / 2
    center_y = (y1 + y2) / 2

    new_width = width * scale_factor
    new_height = height * scale_factor

    new_x1 = center_x - new_width / 2
    new_y1 = center_y - new_height / 2
    new_x2 = center_x + new_width / 2
    new_y2 = center_y + new_height / 2

    return [new_x1, new_y1, new_x2, new_y2]

# Shift bounding boxes slightly to reduce overlap
def shift_bbox(bbox, shift_value=2):
    x1, y1, x2, y2 = bbox
    return [x1 + shift_value, y1 + shift_value, x2 + shift_value, y2 + shift_value]

# Adjust BBoxes in a frame to minimize overlap
def adjust_frame_bboxes(frame_boxes):
    adjusted_boxes = []
    for i in range(len(frame_boxes)):
        for j in range(i + 1, len(frame_boxes)):
            if iou(frame_boxes[i], frame_boxes[j]) > 0:
                # If overlapping, apply a shift
                frame_boxes[j] = shift_bbox(frame_boxes[j])
        adjusted_boxes.append(scale_bbox(frame_boxes[i]))
    return adjusted_boxes

# Mutate by scaling and shifting bounding boxes to avoid overlaps
def mutate(individual, mutation_rate=0.05, max_shift=3, max_bbox_shift=3):
    mutated = deepcopy(individual)
    frames = {}

    for track in mutated:
        for obj in track:
            frame = obj['Frame']
            if frame not in frames:
                frames[frame] = []
            frames[frame].append(obj['Bounding Box'])

    # Adjust BBoxes in each frame to avoid overlap
    for frame, boxes in frames.items():
        frames[frame] = adjust_frame_bboxes(boxes)

    # Apply adjusted BBoxes back to tracks
    for track in mutated:
        for obj in track:
            frame = obj['Frame']
            obj['Bounding Box'] = frames[frame].pop(0)

    return mutated

# Simulated Annealing with track shifting and enhanced mutation
def simulated_annealing(database, initial_temperature=1000, cooling_rate=0.995, mutation_rate=0.1, max_generations=100):
    # Initialize population with a single solution
    current_solution = deepcopy(database)

    current_fitness = fitness(current_solution)
    temperature = initial_temperature

    for generation in range(max_generations):
        # Mutate the current solution
        new_solution = mutate(current_solution, mutation_rate)
        new_fitness = fitness(new_solution)

        # Calculate the acceptance probability
        if new_fitness > current_fitness:
            current_solution = new_solution
            current_fitness = new_fitness
        else:
            acceptance_probability = math.exp((new_fitness - current_fitness) / temperature)
            if random.random() < acceptance_probability:
                current_solution = new_solution
                current_fitness = new_fitness

        # Cool down the temperature
        temperature *= cooling_rate
        if generation % 50 == 0 or generation == max_generations - 1:
            print(f"Generation {generation}, Fitness: {current_fitness:.4f}, Temperature: {temperature:.2f}")

    return current_solution

# Main script to load the CSV and run simulated annealing
if __name__ == "__main__":
    csv_file = '/content/object_tracks.csv'  # Replace with the path to your CSV file
    tracking_database = load_tracking_database(csv_file)

    # Group the tracks by Track ID
    grouped_tracks = group_tracks_by_id(tracking_database)

    # Run simulated annealing on the grouped tracks
    best_tracks = simulated_annealing(grouped_tracks)

    # Flatten the list of tracks for output
    optimized_data = []
    for track in best_tracks:
        optimized_data.extend(track)

    # Convert back to DataFrame for saving or further processing
    optimized_df = pd.DataFrame(optimized_data)
    optimized_df.to_csv('/content/optimized_person_tracks.csv', index=False)

    print("Optimized Person Tracking Data Saved to 'optimized_person_tracks.csv'")
    print(optimized_df.head())


Generation 0, Fitness: 314.3397, Temperature: 995.00
Generation 50, Fitness: 337.2387, Temperature: 774.42
Generation 99, Fitness: 337.2387, Temperature: 605.77
Optimized Person Tracking Data Saved to 'optimized_person_tracks.csv'
   Track ID Class Name  Frame  \
0         1     person    541   
1         1     person    542   
2         1     person    543   
3         1     person    544   
4         1     person    545   

                                        Bounding Box         Hue  Saturation  \
0  [527.47, 1294.3000000000002, 527.47, 1294.3000...  137.394807   71.200518   
1  [528.7249999999999, 1292.0499999999997, 528.72...  136.710455   72.908535   
2  [529.7449999999999, 1292.1999999999998, 529.74...  135.773898   73.498222   
3  [530.7149999999999, 1292.35, 530.7150000000001...  135.370412   75.178289   
4  [530.6199999999999, 1293.1999999999998, 530.62...  135.139405   76.280572   

       Value                           Crop Path  
0  79.612332  object_crops/track_1_fra

In [ ]:
import numpy as np
import pandas as pd
import random
import math
from copy import deepcopy

# Load database from CSV file with additional object information
def load_tracking_database(csv_file):
    df = pd.read_csv(csv_file)
    # Clean up the 'Bounding Box' column and convert to list of coordinates
    # The original code assumed comma-separated values and tried to convert them to int.
    # This is updated to handle space-separated float values enclosed in brackets.
    df['Bounding Box'] = df['Bounding Box'].apply(lambda x: [float(coord) for coord in x.strip('[]').split()])

    # Filter to only include persons
    df = df[df['Class Name'] == 'person']

    # Convert dataframe to list of dictionaries
    database = df.to_dict(orient='records')
    return database

# Group the tracking data by Track ID
def group_tracks_by_id(database):
    grouped_tracks = {}
    for record in database:
        track_id = record['Track ID']
        if track_id not in grouped_tracks:
            grouped_tracks[track_id] = []
        grouped_tracks[track_id].append(record)
    return list(grouped_tracks.values())

# Utility to compute Intersection over Union (IoU)
def iou(box1, box2):
    x1, y1, x2, y2 = box1
    x1_b, y1_b, x2_b, y2_b = box2

    # Calculate intersection
    inter_x1 = max(x1, x1_b)
    inter_y1 = max(y1, y1_b)
    inter_x2 = min(x2, x2_b)
    inter_y2 = min(y2, y2_b)

    inter_area = max(0, inter_x2 - inter_x1) * max(0, inter_y2 - inter_y1)
    box1_area = (x2 - x1) * (y2 - y1)
    box2_area = (x2_b - x1_b) * (y2_b - y1_b)
    union_area = box1_area + box2_area - inter_area

    return inter_area / union_area if union_area != 0 else 0

# Temporal coherence: smoothness of bounding box motion over frames
def temporal_coherence(tracks):
    coherence_score = 0
    for track in tracks:
        for i in range(1, len(track)):
            box_prev = track[i-1]['Bounding Box']
            box_curr = track[i]['Bounding Box']
            displacement = np.linalg.norm(np.array(box_prev[:2]) - np.array(box_curr[:2]))
            coherence_score += np.exp(-displacement)
    return coherence_score

# Spatial coherence: minimize overlap between bounding boxes in the same frame
def spatial_coherence(tracks):
    overlap_penalty = 0
    frames = {}
    for track in tracks:
        for frame_info in track:
            frame = frame_info['Frame']
            box = frame_info['Bounding Box']
            if frame not in frames:
                frames[frame] = []
            frames[frame].append(box)

    for boxes in frames.values():
        for i in range(len(boxes)):
            for j in range(i + 1, len(boxes)):
                overlap_penalty += max(0, iou(boxes[i], boxes[j]) - 0.1)  # Penalize for high overlap

    return overlap_penalty

# Fitness function with temporal and spatial coherence
def fitness(tracks, w1=1.5, w2=2):
    return w1 * temporal_coherence(tracks) - w2 * spatial_coherence(tracks)

# Calculate area of a bounding box
def calculate_area(bbox):
    x1, y1, x2, y2 = bbox
    return (x2 - x1) * (y2 - y1)

# Calculate velocity based on consecutive frames for each track
def calculate_velocity(track):
    prev_frame, prev_center = None, None
    for obj in track:
        current_bbox = obj['Bounding Box']
        current_frame = obj['Frame']

        # Calculate center of the current bounding box
        center_x = (current_bbox[0] + current_bbox[2]) / 2
        center_y = (current_bbox[1] + current_bbox[3]) / 2

        if prev_frame is not None and current_frame == prev_frame + 1:
            # Calculate velocity if it's the next frame in the sequence
            velocity = math.sqrt((center_x - prev_center[0]) ** 2 + (center_y - prev_center[1]) ** 2)
        else:
            # First frame or non-consecutive frames
            velocity = 0

        # Store velocity and area in the object
        obj['Area'] = calculate_area(current_bbox)
        obj['Velocity'] = velocity

        # Update previous frame and center
        prev_frame = current_frame
        prev_center = (center_x, center_y)

    return track

# Scale down the bounding box to reduce chances of overlap
def scale_bbox(bbox, scale_factor=0.5):
    x1, y1, x2, y2 = bbox
    width = x2 - x1
    height = y2 - y1
    center_x = (x1 + x2) / 2
    center_y = (y1 + y2) / 2

    new_width = width * scale_factor
    new_height = height * scale_factor

    new_x1 = center_x - new_width / 2
    new_y1 = center_y - new_height / 2
    new_x2 = center_x + new_width / 2
    new_y2 = center_y + new_height / 2

    return [new_x1, new_y1, new_x2, new_y2]

# Shift bounding boxes slightly to reduce overlap
def shift_bbox(bbox, shift_value=2):
    x1, y1, x2, y2 = bbox
    return [x1 + shift_value, y1 + shift_value, x2 + shift_value, y2 + shift_value]

# Adjust BBoxes in a frame to minimize overlap
def adjust_frame_bboxes(frame_boxes):
    adjusted_boxes = []
    for i in range(len(frame_boxes)):
        for j in range(i + 1, len(frame_boxes)):
            if iou(frame_boxes[i], frame_boxes[j]) > 0:
                # If overlapping, apply a shift
                frame_boxes[j] = shift_bbox(frame_boxes[j])
        adjusted_boxes.append(scale_bbox(frame_boxes[i]))
    return adjusted_boxes

# Mutate by scaling and shifting bounding boxes to avoid overlaps
def mutate(individual, mutation_rate=0.05, max_shift=3, max_bbox_shift=3):
    mutated = deepcopy(individual)
    frames = {}

    for track in mutated:
        for obj in track:
            frame = obj['Frame']
            if frame not in frames:
                frames[frame] = []
            frames[frame].append(obj['Bounding Box'])

    # Adjust BBoxes in each frame to avoid overlap
    for frame, boxes in frames.items():
        frames[frame] = adjust_frame_bboxes(boxes)

    # Apply adjusted BBoxes back to tracks
    for track in mutated:
        for obj in track:
            frame = obj['Frame']
            obj['Bounding Box'] = frames[frame].pop(0)

    return mutated

# Simulated Annealing with track shifting and enhanced mutation
def simulated_annealing(database, initial_temperature=1000, cooling_rate=0.995, mutation_rate=0.1, max_generations=100):
    # Initialize population with a single solution
    current_solution = deepcopy(database)

    # Calculate area and velocity for the initial solution
    current_solution = [calculate_velocity(track) for track in current_solution]
    current_fitness = fitness(current_solution)
    temperature = initial_temperature

    for generation in range(max_generations):
        # Mutate the current solution
        new_solution = mutate(current_solution, mutation_rate)

        # Calculate area and velocity for the new solution
        new_solution = [calculate_velocity(track) for track in new_solution]
        new_fitness = fitness(new_solution)

        # Calculate the acceptance probability
        if new_fitness > current_fitness:
            current_solution = new_solution
            current_fitness = new_fitness
        else:
            acceptance_probability = math.exp((new_fitness - current_fitness) / temperature)
            if random.random() < acceptance_probability:
                current_solution = new_solution
                current_fitness = new_fitness

        # Cool down the temperature
        temperature *= cooling_rate
        if generation % 50 == 0 or generation == max_generations - 1:
            print(f"Generation {generation}, Fitness: {current_fitness:.4f}, Temperature: {temperature:.2f}")

    return current_solution

# Main script to load the CSV and run simulated annealing
if __name__ == "__main__":
    csv_file = '/content/object_tracks.csv'  # Replace with the path to your CSV file
    tracking_database = load_tracking_database(csv_file)

    # Group the tracks by Track ID
    grouped_tracks = group_tracks_by_id(tracking_database)

    # Run simulated annealing on the grouped tracks
    best_tracks = simulated_annealing(grouped_tracks)

    # Flatten the list of tracks for output
    optimized_data = []
    for track in best_tracks:
        optimized_data.extend(track)

    # Convert back to DataFrame for saving or further processing
    optimized_df = pd.DataFrame(optimized_data)

    # Save optimized data with area and velocity included
    optimized_df.to_csv('/content/optimized_person_tracks1.csv', index=False)

    print("Optimized Person Tracking Data Saved to 'optimized_person_tracks.csv'")
    print(optimized_df.head())


Generation 0, Fitness: 314.3397, Temperature: 995.00
Generation 50, Fitness: 337.2387, Temperature: 774.42
Generation 99, Fitness: 337.2387, Temperature: 605.77
Optimized Person Tracking Data Saved to 'optimized_person_tracks.csv'
   Track ID Class Name  Frame  \
0         1     person    541   
1         1     person    542   
2         1     person    543   
3         1     person    544   
4         1     person    545   

                                        Bounding Box         Hue  Saturation  \
0  [527.47, 1294.3000000000002, 527.47, 1294.3000...  137.394807   71.200518   
1  [528.7249999999999, 1292.0499999999997, 528.72...  136.710455   72.908535   
2  [529.7449999999999, 1292.1999999999998, 529.74...  135.773898   73.498222   
3  [530.7149999999999, 1292.35, 530.7150000000001...  135.370412   75.178289   
4  [530.6199999999999, 1293.1999999999998, 530.62...  135.139405   76.280572   

       Value                           Crop Path          Area  Velocity  
0  79.612332  

In [53]:
import cv2
import csv
import os
import numpy as np
from tqdm import tqdm

def generate_video_synopsis(bg_image_path, csv_file, crops_dir, output_dir, video_output=None, fps=30, max_objects_per_frame=5):
    # Load the background image
    bg_image = cv2.imread(bg_image_path)
    if bg_image is None:
        raise ValueError(f"Background image {bg_image_path} not found.")

    # Get dimensions of the background
    frame_height, frame_width, _ = bg_image.shape

    # Initialize video writer if needed
    if video_output:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        video_writer = cv2.VideoWriter(video_output, fourcc, fps, (frame_width, frame_height))

    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Load tracking data from CSV file
    frame_dict = {}
    with open(csv_file, mode='r') as file:
        csv_reader = csv.DictReader(file)
        for row in csv_reader:
            frame_num = int(row['Frame'])
            track_id = row['Track ID']
            class_name = row['Class Name']
            bbox = eval(row['Bounding Box'])  # Format: (x1, y1, x2, y2)
            crop_path = os.path.join(crops_dir, row['Crop Path'])

            # Organize data per frame
            if frame_num not in frame_dict:
                frame_dict[frame_num] = []
            frame_dict[frame_num].append((track_id, class_name, bbox, crop_path))

    # Object data structure: track ID -> list of frame info (bbox, crop path)
    track_data = {}
    for frame_num in sorted(frame_dict.keys()):
        for track_id, class_name, bbox, crop_path in frame_dict[frame_num]:
            if track_id not in track_data:
                track_data[track_id] = []
            track_data[track_id].append((frame_num, bbox, crop_path))

    # Step 2: Video synopsis by blending objects from different times
    synopsis_frames = []
    current_frame_num = 0
    used_tracks = set()

    while len(used_tracks) < len(track_data):
        # Create a new frame by cloning the background
        frame_image = bg_image.copy()

        # Select objects to display in this synopsis frame
        objects_in_frame = 0
        for track_id, track_frames in track_data.items():
            if track_id in used_tracks:
                continue

            # Find the next available frame for this track
            for frame_info in track_frames:
                frame_num, bbox, crop_path = frame_info
                if objects_in_frame >= max_objects_per_frame:
                    break  # Max number of objects reached for this frame

                # Blend object into the frame
                x1, y1, x2, y2 = bbox
                if x2 <= x1 or y2 <= y1:
                    continue  # Skip invalid bounding boxes

                # Load the cropped object image
                crop_img = cv2.imread(crop_path)
                if crop_img is None:
                    continue

                crop_height, crop_width, _ = crop_img.shape
                centroid_x = (x1 + x2) // 2
                centroid_y = (y1 + y2) // 2

                top_left_x = int(centroid_x - (crop_width // 2))
                top_left_y = int(centroid_y - (crop_height // 2))

                top_left_x = max(0, min(top_left_x, frame_width - crop_width))
                top_left_y = max(0, min(top_left_y, frame_height - crop_height))

                try:
                    roi = frame_image[top_left_y:top_left_y + crop_height, top_left_x:top_left_x + crop_width]
                    crop_gray = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
                    _, mask = cv2.threshold(crop_gray, 1, 255, cv2.THRESH_BINARY)
                    mask_inv = cv2.bitwise_not(mask)
                    bg_part = cv2.bitwise_and(roi, roi, mask=mask_inv)
                    fg_part = cv2.bitwise_and(crop_img, crop_img, mask=mask)
                    dst = cv2.add(bg_part, fg_part)
                    frame_image[top_left_y:top_left_y + crop_height, top_left_x:top_left_x + crop_width] = dst
                    objects_in_frame += 1
                    used_tracks.add(track_id)
                except Exception as e:
                    print(f"Error blending object in synopsis frame {current_frame_num}, track {track_id}: {str(e)}")


        # Optionally write the frame to the video
        if video_output:
            video_writer.write(frame_image)

        current_frame_num += 1


    # Release video writer if it was used
    if video_output:
        video_writer.release()

    print(f"Video synopsis complete. Frames saved to {output_dir}.")
    if video_output:
        print(f"Video saved to {video_output}.")

# Example usage
bg_image_path = "/content/background.jpg"
csv_file = "/content/optimized_person_tracks.csv"
crops_dir = "/content"
output_dir = "synopsis_frames"
video_output = "synopsis_video.mp4"  # Set to None if you don't need a video output

generate_video_synopsis(bg_image_path, csv_file, crops_dir, output_dir, video_output=video_output, fps=5, max_objects_per_frame=20)


Video synopsis complete. Frames saved to synopsis_frames.
Video saved to synopsis_video.mp4.


In [54]:
!ffmpeg -hide_banner -loglevel error -i /content/synopsis_video.mp4 -vcodec libx264 /content/output_final.mp4 -y
play_video("/content/output_final.mp4")

In [52]:
!rm -rf /content/synopsis_frames
!rm -rf /content/synopsis_video.mp4
!rm -rf /content/output_final.mp4